This notebook explores a few different potential models of the Milky Way

In [ ]:
using CairoMakie
using Revise
using LilGuys, Arya

In [ ]:
using PythonCall
agama = pyimport("agama")

In [ ]:
using Printf

In [ ]:
include("agama_utils.jl")

In [ ]:
lmc_coord = ICRS(ra = 80.8,
    dec = -69.8, 
    pmra = 1.910,
    pmdec = 0.229,
    radial_velocity = 262,
    distance = 49.59
    )

lmc_coord_err = ICRS(
    ra=0.3,
    dec=0.3,
    distance=0.54,
    pmra=0.2,
    pmdec=0.47,
    radial_velocity=3,
    )

In [ ]:
gc = LilGuys.transform(Galactocentric, lmc_coord)

In [ ]:
ep20 = load_agama_potential("EP2020.ini")


In [ ]:
function leap_frog(gc, acceleration; dt_max=0.1, dt_min=0.001, time=-10/T2GYR, timestep=:adaptive, η=0.01)

    Nt = round(Int, abs(time / dt_min))
    positions = Vector{Vector{Float64}}()
    velocities = Vector{Vector{Float64}}()
    times = Float64[]

    push!(positions, [gc.x, gc.y, gc.z])
    push!(velocities, [gc.v_x, gc.v_y, gc.v_z] / V2KMS)
    push!(times, 0.)
    is_done = false
    backwards = time < 0

    t = 0.
    for i in 1:Nt
        pos = positions[i]
        vel = velocities[i]
        acc = acceleration(pos, vel)

        dt = min(sqrt(η / calc_r(acc)), dt_max)
        
        if backwards
            dt *= -1
        end
        if abs(dt) < dt_min
            @warn "timestep below minimum timestep"
            break
        end

        if (backwards && t + dt <= time ) || (!backwards && t + dt >= time)
            dt = time - t
            is_done = true
        end

        vel_h = vel + 1/2 * dt * acc
        pos_new = pos + dt*vel_h
        acc = acceleration(pos_new, vel_h)
        vel_new = vel_h + 1/2 * dt * acc

        push!(positions, pos_new)
        push!(velocities, vel_new)
        t = times[i] + dt
        push!(times, t)

        if is_done
            break
        end
    end

    positions_matrix = hcat(positions...)
    velocities_matrix = hcat(velocities...)
    return Orbit(time=times, position=positions_matrix, velocity=velocities_matrix)
end

In [ ]:
ep20.force

In [ ]:
function calc_σv_interp(pot; log_r = LinRange(5, -3, 100000))
    x0 = [1/√2, 0., 1/√2] # direction

    radii = 10 .^ log_r
    positions = x0' .* radii
    acc = pyconvert(Matrix{Float64}, pot.force(positions))
    rho = pyconvert(Array{Float64}, pot.density(positions))

    a = calc_r(acc')
    
    dr = abs.(LilGuys.gradient(radii))

    σ2 = 1 ./ rho .* cumsum(rho .* a .* dr)

    return LilGuys.lerp(radii, sqrt.(σ2))
end

In [ ]:
using SpecialFunctions: erf

In [ ]:
function a_dyn_friction(pos, vel; r_s, σv, ρ, M)
    G = 1
    v = calc_r(vel)
    r = calc_r(pos)
    
    if r_s < 8
        ϵ = 0.45r_s
    else
        ϵ = 2.2r_s - 14
    end

    Λ = r / ϵ
    X = v / (√2 * σv(r))
    return -4π * G^2 * M * ρ(pos) * log(Λ) * vel ./ v^3 * ( erf(X) - 2*X/√π * exp(-X^2) )
end

In [ ]:
σv = calc_σv_interp(ep20)

In [ ]:
lines(log10.(σv.x), σv.y * V2KMS)

In [ ]:
lines(log10.(σv.x), σv.y * V2KMS)

In [ ]:
orbit_nofric = calc_orbit(gc, ep20, time=-10/T2GYR)

In [ ]:
f_fric(pos, vel) = a_dyn_friction(pos, vel, r_s=10, σv=σv, ρ=x->pyconvert(Float64, ep20.density(x)), M=10)

In [ ]:
f_acc(pos, vel) = pyconvert(Vector{Float64}, ep20.force(pos)) .+ f_fric(pos, vel)

In [ ]:
orbit_2 = leap_frog(gc, f_acc, dt_max=1)

In [ ]:
fig, ax, _ = plot_r_t(orbit_nofric)
plot_r_t!(ax, orbit_2)

fig

In [ ]:
fig, ax, p = plot_y_z(orbit_nofric)
lines!(orbit_2.position[2, :], orbit_2.position[3, :])

fig


## Adding a galaxy...

In [ ]:
obs_props_filename = ENV["DWARFS_ROOT"] * "/observations/sculptor/observed_properties.toml"

In [ ]:
icrs = LilGuys.coord_from_file(obs_props_filename)
icrs_err = LilGuys.coord_err_from_file(obs_props_filename)

In [ ]:
gc = LilGuys.transform(Galactocentric, icrs)

In [ ]:
ep20 = load_agama_potential("EP2020.ini")


In [ ]:
pot = load_agama_potential("EP2020.ini")

In [ ]:
orbit = calc_orbit(gc, ep20, units=:code, time=-10)

In [ ]:
plot_r_t(orbit)

In [ ]:
plot_y_z(orbit)

In [ ]:
plot_v_circ(ep20)

In [ ]:
mm11 = load_agama_potential("mcmillan11.ini")
mm17 = load_agama_potential("mcmillan17.ini")
bt08 = load_agama_potential("BT08.ini")
piffl14 = load_agama_potential("piffl_14.ini")
J95 = load_agama_potential("J95.ini")
galpy14 = load_agama_potential("galpy_2014.ini")

In [ ]:
potentials_light = [
    "EP20" => ep20,
    "bt08" => bt08,
    "galpy14" => galpy14,
    ]


In [ ]:
potentials_heavy = [
    "EP20" => ep20,
    "mcmillan 11" => mm11,
    "piffl14" => piffl14,
    "J95" => J95,

]

In [ ]:
potentials = [
    "EP20" => ep20,
    "mcmillan 17" => mm17,

]

In [ ]:
my_potentials = {
    "fiducial": EP2020,
    "light": light_mw,
    "heavy": heavy_mw,
    "flat": me_flat,
    "steep": me_steep
}

In [ ]:
for label, pot in my_potentials.items():
    print(label, o.jr(pot))

In [ ]:
o.jr(galpy14)

In [ ]:
o.jr(galpy14_heavy)

In [ ]:
o.jr(EP2020)

In [ ]:
o.jr(ablimit20a)

In [ ]:
o.jr(mcmillan2017)

In [ ]:
for label, pot in my_potentials.items():
    print(label, o.jz(pot))

In [ ]:
o.jz(galpy14)

In [ ]:
o.jz(galpy14_heavy)

In [ ]:
o.jz(EP2020)

In [ ]:
o.jz(mcmillan2017)

In [ ]:
plot_Vc(EP2020_bulge)
plot_Vc(mcmillan2011_bulge)
plot_Vc(galpy14_bulge)
plot_Vc(pouliasis17_bulge)

In [ ]:
plot_Vc(EP2020_thick + EP2020_thick)
plot_Vc(galpy14_disk)
# plot_Vc(galpy14_disk)

plot_Vc(pouliasis17_thick + pouliasis17_thick)
# plot_Vc(pouliasis17b_thick + pouliasis17b_thick)

In [ ]:
plot_Vc(EP2020_halo)
plot_Vc(mcmillan2011_halo)
plot_Vc(mcmillan2017_halo)
plot_Vc(galpy14_halo)
plot_Vc(2*galpy14_halo)

plot_Vc(pouliasis17_halo)
plot_Vc(pouliasis17b_halo)
plot_Vc(ablimit_b_comp["halo"]) # pretty much same as galpy


In [ ]:
dwarf_galaxy_gc.icrs

In [ ]:
o = Orbit(dwarf_galaxy_gc)

In [ ]:
pot = EP2020
ts = np.linspace(0, -5, 10_000) * u.Gyr
o.integrate(ts, pot)

In [ ]:
fig, ax = plt.subplots()

plt.plot(o.R(ts), o.z(ts))
ax.set_aspect(1)

In [ ]:
fig, ax = plt.subplots()

plt.plot(o.y(ts), o.z(ts))

ax.set_aspect(1)

In [ ]:
plt.plot(ts, o.r(ts))

In [ ]:
pos_gc = []

ts = np.linspace(0, 0.1, 10_000) * u.Gyr
o.integrate(ts, pot)

# galpy uses left-handed frame :/
pos_gc = coord.SkyCoord(
    x=- o.x(ts), y=o.y(ts), z=o.z(ts), 
    v_x = -o.vx(ts), v_y=o.vy(ts), v_z=o.vz(ts),
    frame=gc_frame)

pos_icrs = pos_gc.transform_to("icrs")

In [ ]:
plt.scatter(pos_icrs.ra, pos_icrs.dec, c=ts)
plt.colorbar()

plt.plot(o.ra(ts), o.dec(ts))

plt.scatter(dwarf_galaxy.ra, dwarf_galaxy.dec)


tpm = 3e7*u.yr
plt.plot(sculptor.ra + sculptor.pm_ra_cosdec / np.cos(sculptor.dec) * tpm * np.array([0, 1]),
        sculptor.dec + sculptor.pm_dec * tpm * np.array([0,1]),
         zorder=5, color=arya.COLORS[2]
        )

plt.xlabel("RA")
plt.ylabel("dec")

In [ ]:
sc2 = dwarf_galaxy_gc.transform_to(coord.ICRS)

# LMC Orbit

In [ ]:
lmc_coord = ICRS(ra = 81,
    dec = -69.75, 
    pmra = 1.8,
    pmdec = 0.35,
    radial_velocity = 260,
    distance = 50.
    )

In [ ]:
lmc_gc = LilGuys.transform(LilGuys.Galactocentric, lmc_coord)

In [ ]:
vasiliev21_frozen = load_agama_potential("vasiliev+21/potential_nolmc.ini")


In [ ]:
vasiliev21 = load_agama_potential("vasiliev+21/potential_evolving.ini")


In [ ]:
orbit_lmc_me = calc_orbit(lmc_gc, vasiliev21_frozen, units=:vasiliev, time=LinRange(0, -10, 1000))


In [ ]:
using CSV, DataFrames

In [ ]:
# loads in trajectory of lmc in Vasiliev 2021
lmc_file = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev+21/trajlmc.txt"
lmc_traj = CSV.read(lmc_file, DataFrame, delim=" ", header = [:time, :x, :y, :z, :v_x, :v_y, :v_z])

filt = lmc_traj.time .<= 0
lmc_traj = lmc_traj[filt, :]
lmc_traj = DataFrame(reverse(eachrow(lmc_traj)))


lmc_orbit = Orbit(time=lmc_traj.time  * V_T2GYR/ T2GYR, 
    position = [lmc_traj.x lmc_traj.y lmc_traj.z]',
    velocity = [lmc_traj.v_x lmc_traj.v_y lmc_traj.v_z]' * V_V2KMS / V2KMS
    )



In [ ]:
scl_orbit = calc_orbit(gc, vasiliev21_frozen, units=:vasiliev, time=-5)

In [ ]:
scl_lmc_orbit = calc_orbit(gc, vasiliev21, units=:vasiliev, time=lmc_traj.time) # need to pass vasiliev time to this class...

In [ ]:
orbit_lmc_me = calc_orbit(lmc_gc, vasiliev21_frozen, units=:vasiliev, time=lmc_traj.time)


In [ ]:
scl_m_lmc = deepcopy(scl_lmc_orbit)

scl_m_lmc.position .-= lmc_orbit.position

In [ ]:
r_scl_lmc = calc_r(lmc_orbit.position, scl_lmc_orbit.position)

In [ ]:
LilGuys.Plots.plot_xyz(lmc_orbit.position, scl_lmc_orbit.position, scl_orbit.position, labels=["LMC", "Scl", "Scl (no lmc)"])

In [ ]:
fig, ax, p = plot_y_z(scl_m_lmc)
Colorbar(fig[1, 2], p)
fig

In [ ]:
orbits = [
    "Scl-MW" => scl_lmc_orbit,
    "Scl-MW (no LMC)" => scl_orbit,
    "LMC-MW" => lmc_orbit,
   # "lmc me" => orbit_lmc_me,
    ]

In [ ]:
plot_y_z(orbits)

In [ ]:
V_M2MSUN  / M2MSUN

In [ ]:
T2GYR / V_T2GYR

In [ ]:
V_V2KMS / V2KMS

In [ ]:
fig = plot_r_t(orbits, legend=false)

lines!(lmc_orbit.time * T2GYR, r_scl_lmc, label="Scl-LMC ")
axislegend()
fig

In [ ]:
log10(LilGuys.kpc_to_arcmin(LilGuys.calc_break_radius(8.0 / V2KMS, 0.1 / T2GYR), 83.2))

In [ ]:
log10(20)

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance from Scl to LMC / kpc",
    limits=(nothing, nothing, 0, nothing),
    )

lines!(lmc_orbit.time * T2GYR, r_scl_lmc)

fig

In [ ]:
lmc_orbit.time[argmin(r_scl_lmc)] * T2GYR

In [ ]:
minimum(r_scl_lmc)

In [ ]:
lmc_orbit.time ./ scl_lmc_orbit.time

In [ ]:
plot_v_circ(ep20)

In [ ]:
plot_v_circ(vasiliev21, vasiliev_units=true)

# MC orbits

In [ ]:
include("agama_utils.jl")

In [ ]:
icrs_samples = LilGuys.rand_coords(icrs, icrs_err, 1000);
gc_samples = LilGuys.transform.(Galactocentric, icrs_samples);

In [ ]:
icrs_df = LilGuys.to_frame(icrs_samples)

In [ ]:
ic = make_agama_init(gc_samples)

In [ ]:
lmc_orbit.time

lmc.times

In [ ]:
orbits = calc_orbits(gc_samples, ep20, time=-5 / T2GYR);

In [ ]:
plot_y_z(orbits[1].second)

In [ ]:
plot_r_t(orbits[1:100], alpha=0.2, color=:black, legend=false)

In [ ]:
orbits = calc_orbits(gc_samples, vasiliev21_frozen, time=-5 / V_T2GYR, units=:vasiliev);

In [ ]:
plot_y_z(orbits[1].second)

In [ ]:
plot_r_t(orbits[1:100], alpha=0.2, color=:black, legend=false)

In [ ]:
orbits = calc_orbits(gc_samples, vasiliev21, time=lmc_traj.time, units=:vasiliev);

In [ ]:
plot_r_t(orbits[1:100], alpha=0.2, color=:black, legend=false)

In [ ]:
rs = [calc_r(lmc_orbit.position, o.position) for (l, o) in orbits];

In [ ]:
rs2 = [calc_r(orbit_lmc_me.position, o.position) for (l, o) in orbits];

In [ ]:
scl_lmc_times = lmc_orbit.time[argmin.(rs)] * T2GYR

In [ ]:
scl_lmc_mins = minimum.(rs)

In [ ]:
hist(scl_lmc_times)

In [ ]:
hist(scl_lmc_mins)

In [ ]:
scatter(icrs_df.pmra, scl_lmc_mins)

In [ ]:
scatter(icrs_df.pmdec, scl_lmc_mins)

In [ ]:
scatter(icrs_df.distance, scl_lmc_mins)

In [ ]:
scatter(icrs_df.distance, scl_lmc_times)

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance from Scl to LMC / kpc",
    limits=(nothing, nothing, 0, nothing),
    )


for r in rs
    lines!(lmc_orbit.time * T2GYR, r, alpha=0.05, color=:black)
end

fig

In [ ]:
orbits[1].second.position

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance  / kpc",
    limits=(nothing, nothing, 0, nothing),
    )


lines!(lmc_orbit.time * T2GYR, rs[1], label="Scl - LMC")
lines!(lmc_orbit.time * T2GYR, rs2[1], label="Scl - LMC point particle")

lines!(orbits[1].second.time * T2GYR, calc_r(orbits[1].second.position), label="Scl - MW")

axislegend(position=:lb)

fig